# Eat Safe, Love

## Part 1: Database and Jupyter Notebook Set Up

To create the database: 
1. Navigate to the correct folder on your local drive in your terminal. Type in: 
mongoimport --type json -d uk_food -c establishments --drop --jsonArray establishments.json

This creates a database named uk_food and a collection named establishments. 



In [41]:
# Import dependencies
from pymongo import MongoClient
from pprint import pprint



In [42]:
# Create an instance of MongoClient
mongo = MongoClient(port=27017)

In [43]:
# confirm that our new database was created
print(mongo.list_database_names())


['admin', 'autosaurus', 'classDB', 'config', 'fruits_db', 'garden_db', 'local', 'met', 'pesitly_marketing', 'petsitly_marketing', 'travel_db', 'uk_food']


In [44]:
# assign the uk_food database to a variable name
db = mongo['uk_food']

In [45]:
# review the collections in our new database
print(db.list_collection_names())

['establishments']


In [46]:
# review a document in the establishments collection
pprint(db.establishments.find_one())


{'AddressLine1': '1 The Droveway',
 'AddressLine2': 'St Margarets Bay',
 'AddressLine3': 'Kent',
 'AddressLine4': '',
 'BusinessName': 'FirstLight Bar & Café',
 'BusinessType': 'Restaurant/Cafe/Canteen',
 'BusinessTypeID': 1,
 'ChangesByServerID': 0,
 'Distance': 4587.64041272672,
 'FHRSID': 1290215,
 'LocalAuthorityBusinessID': 'PI/000083029',
 'LocalAuthorityCode': '182',
 'LocalAuthorityEmailAddress': 'publicprotection@dover.gov.uk',
 'LocalAuthorityName': 'Dover',
 'LocalAuthorityWebSite': 'http://www.dover.gov.uk/',
 'NewRatingPending': False,
 'Phone': '',
 'PostCode': 'CT15 6DH',
 'RatingDate': '2020-11-09T00:00:00',
 'RatingKey': 'fhrs_5_en-gb',
 'RatingValue': '5',
 'RightToReply': '',
 'SchemeType': 'FHRS',
 '_id': ObjectId('63d6e7aca6ce35fc05daf851'),
 'geocode': {'latitude': '51.150625', 'longitude': '1.378231'},
 'links': [{'href': 'https://api.ratings.food.gov.uk/establishments/1290215',
            'rel': 'self'}],
 'meta': {'dataSource': None,
          'extractDate': '

In [47]:
# assign the collection to a variable
establishments = db['establishments']

## Part 2: Update the Database

1. An exciting new halal restaurant just opened in Greenwich, but hasn't been rated yet. The magazine has asked you to include it in your analysis. Add the following restaurant "Penang Flavours" to the database.

In [48]:
# Create a dictionary for the new restaurant data
#Because I do not have any other data on the restaurant, I am only adding in fields for which I have data
penang_flavors = {
    "BusinessName":"Penang Flavours",
    "BusinessType":"Restaurant/Cafe/Canteen",
    "BusinessTypeID":"",
    "AddressLine1":"Penang Flavours",
    "AddressLine2":"146A Plumstead Rd",
    "AddressLine3":"London",
    "AddressLine4":"",
    "PostCode":"SE18 7DY",
    "Phone":"",
    "LocalAuthorityCode":"511",
    "LocalAuthorityName":"Greenwich",
    "LocalAuthorityWebSite":"http://www.royalgreenwich.gov.uk",
    "LocalAuthorityEmailAddress":"health@royalgreenwich.gov.uk",
    "scores":{
        "Hygiene":"",
        "Structural":"",
        "ConfidenceInManagement":""
    },
    "SchemeType":"FHRS",
    "geocode":{
        "longitude":"0.08384000",
        "latitude":"51.49014200"
    },
    "RightToReply":"",
    "Distance":4623.9723280747176,
    "NewRatingPending":True
}



In [49]:
# Insert the new restaurant into the collection
establishments.insert_one(penang_flavors)

In [50]:
# Check that the new restaurant was inserted
pprint(establishments.find_one({'BusinessName':'Penang Flavours'}))

{'AddressLine1': 'Penang Flavours',
 'AddressLine2': '146A Plumstead Rd',
 'AddressLine3': 'London',
 'AddressLine4': '',
 'BusinessName': 'Penang Flavours',
 'BusinessType': 'Restaurant/Cafe/Canteen',
 'BusinessTypeID': '',
 'Distance': 4623.972328074718,
 'LocalAuthorityCode': '511',
 'LocalAuthorityEmailAddress': 'health@royalgreenwich.gov.uk',
 'LocalAuthorityName': 'Greenwich',
 'LocalAuthorityWebSite': 'http://www.royalgreenwich.gov.uk',
 'NewRatingPending': True,
 'Phone': '',
 'PostCode': 'SE18 7DY',
 'RightToReply': '',
 'SchemeType': 'FHRS',
 '_id': ObjectId('63d6e7c46d8e788fcf33436c'),
 'geocode': {'latitude': '51.49014200', 'longitude': '0.08384000'},
 'scores': {'ConfidenceInManagement': '', 'Hygiene': '', 'Structural': ''}}


2. Find the BusinessTypeID for "Restaurant/Cafe/Canteen" and return only the `BusinessTypeID` and `BusinessType` fields.

In [51]:
# Find the BusinessTypeID for "Restaurant/Cafe/Canteen" and return only the BusinessTypeID and BusinessType fields

#the query variable defines the field : value I am looking for
query = {'BusinessType' : 'Restaurant/Cafe/Canteen'}

#the fields variable flags the fields as TRUE to return in the query
fields = {'BusinessType': 1, 'BusinessTypeID': 1}

#results uses the query and fields variables to run the query and return the fields I am interested in
results = db.establishments.find(query, fields)

pprint(results[0])


{'BusinessType': 'Restaurant/Cafe/Canteen',
 'BusinessTypeID': 1,
 '_id': ObjectId('63d6e7aca6ce35fc05daf851')}


3. Update the new restaurant with the `BusinessTypeID` you found.

In [52]:
# Update the new restaurant with the correct BusinessTypeID

#create a variable for finding the restaurant
query = {'BusinessName': 'Penang Flavours'}

#create a variable for adding the field to the record
fields = {'$set': {'BusinessTypeID': 1}}

#use update_one to update the restaurant's record
update = db.establishments.update_one(query, fields)


In [53]:
# Confirm that the new restaurant was updated
query = {'BusinessName': 'Penang Flavours'}
result = db.establishments.find_one(query)

pprint(result)

{'AddressLine1': 'Penang Flavours',
 'AddressLine2': '146A Plumstead Rd',
 'AddressLine3': 'London',
 'AddressLine4': '',
 'BusinessName': 'Penang Flavours',
 'BusinessType': 'Restaurant/Cafe/Canteen',
 'BusinessTypeID': 1,
 'Distance': 4623.972328074718,
 'LocalAuthorityCode': '511',
 'LocalAuthorityEmailAddress': 'health@royalgreenwich.gov.uk',
 'LocalAuthorityName': 'Greenwich',
 'LocalAuthorityWebSite': 'http://www.royalgreenwich.gov.uk',
 'NewRatingPending': True,
 'Phone': '',
 'PostCode': 'SE18 7DY',
 'RightToReply': '',
 'SchemeType': 'FHRS',
 '_id': ObjectId('63d6e7c46d8e788fcf33436c'),
 'geocode': {'latitude': '51.49014200', 'longitude': '0.08384000'},
 'scores': {'ConfidenceInManagement': '', 'Hygiene': '', 'Structural': ''}}


4. The magazine is not interested in any establishments in Dover, so check how many documents contain the Dover Local Authority. Then, remove any establishments within the Dover Local Authority from the database, and check the number of documents to ensure they were deleted.

In [54]:
# Find how many documents have LocalAuthorityName as "Dover"
query = {'LocalAuthorityName': 'Dover'}

print("Number of documents with LocalAuthorityName as 'Dover':", establishments.count_documents(query))

Number of documents with LocalAuthorityName as 'Dover': 994


In [55]:
# Delete all documents where LocalAuthorityName is "Dover"
query = {'LocalAuthorityName': 'Dover'}

delete = db.establishments.delete_many(query)


In [56]:
# Check if any remaining documents include Dover
query = {'LocalAuthorityName': 'Dover'}

print("Number of documents with LocalAuthorityName as 'Dover':", establishments.count_documents(query))

Number of documents with LocalAuthorityName as 'Dover': 0


In [57]:
# Check that other documents remain with 'find_one'
fields = {'LocalAuthorityName':1}
query = {}

results = db.establishments.find_one(query, fields)

pprint(results)

{'LocalAuthorityName': 'Folkestone and Hythe',
 '_id': ObjectId('63d6e7aca6ce35fc05dafb36')}


5. Some of the number values are stored as strings, when they should be stored as numbers. Use `update_many` to convert `latitude` and `longitude` to decimal numbers.

In [58]:
# Change the data type from String to Decimal for longitude
query = {}
type_change = [{'$set':{ "geocode.longitude" : {'$toDouble': "$geocode.longitude"}}}]
update = db.establishments.update_many(query, type_change)

#verify
fields = {'geocode.latitude':1, 'geocode.longitude':1}
pprint(db.establishments.find_one(query, fields))

{'_id': ObjectId('63d6e7aca6ce35fc05dafb36'),
 'geocode': {'latitude': '51.083812', 'longitude': 1.195625}}


In [59]:
# Change the data type from String to Decimal for latitude
query = {}
type_change = [{'$set':{ "geocode.latitude" : {'$toDouble': "$geocode.latitude"}}}]
update = db.establishments.update_many(query, type_change)

#verify
fields = {'geocode.latitude':1, 'geocode.longitude':1}
pprint(db.establishments.find_one(query, fields))

{'_id': ObjectId('63d6e7aca6ce35fc05dafb36'),
 'geocode': {'latitude': 51.083812, 'longitude': 1.195625}}


In [60]:
# Check that the coordinates are now numbers
query = {}
fields = {'geocode.latitude':1, 'geocode.longitude':1}
results = db.establishments.find(query, fields)

pprint(list(results[0:10]))


[{'_id': ObjectId('63d6e7aca6ce35fc05dafb36'),
  'geocode': {'latitude': 51.083812, 'longitude': 1.195625}},
 {'_id': ObjectId('63d6e7aca6ce35fc05dafb37'),
  'geocode': {'latitude': 51.086058, 'longitude': 1.196408}},
 {'_id': ObjectId('63d6e7aca6ce35fc05dafb38'),
  'geocode': {'latitude': 51.085797, 'longitude': 1.194762}},
 {'_id': ObjectId('63d6e7aca6ce35fc05dafb3b'),
  'geocode': {'latitude': 51.08084, 'longitude': 1.188537}},
 {'_id': ObjectId('63d6e7aca6ce35fc05dafb3c'),
  'geocode': {'latitude': 51.08084, 'longitude': 1.188537}},
 {'_id': ObjectId('63d6e7aca6ce35fc05dafb3d'),
  'geocode': {'latitude': 51.08084, 'longitude': 1.188537}},
 {'_id': ObjectId('63d6e7aca6ce35fc05dafb3f'),
  'geocode': {'latitude': 51.08084, 'longitude': 1.188537}},
 {'_id': ObjectId('63d6e7aca6ce35fc05dafb40'),
  'geocode': {'latitude': 51.0783519967076, 'longitude': 1.18590330311705}},
 {'_id': ObjectId('63d6e7aca6ce35fc05dafb41'),
  'geocode': {'latitude': 51.0783519967076, 'longitude': 1.18590330311